In [1]:
import os
import scanpy as sc
import numpy as np
import pandas as pd
import anndata
import random

In [2]:
# Input path of the reference/raw data
PATH_PROJECT = "/mnt/DOSI/PLATEFORMES/BIOINFORMATIQUE/04_PROJECT/scLLM"
PATH_EXPERIMENT = os.path.join( PATH_PROJECT, "cross_tissue_immune_cell")
PATH_EXPERIMENT_REFERENCE = os.path.join( PATH_EXPERIMENT, "01_Reference")
PATH_EXPERIMENT_REFERENCE_EXTRA = os.path.join( PATH_EXPERIMENT_REFERENCE, "00_Dataset")
PATH_EXPERIMENT_OUTPUT = os.path.join( PATH_EXPERIMENT, "05_Output")

PATH_INPUT_FILE = os.path.join( PATH_EXPERIMENT_REFERENCE_EXTRA, "cross_tissue_immune_cell.h5ad")


# Output path of the pre processed dataset
ANALYSIS_NAME = "01_Datapreprocessing"
EXTRA_ANALYSIS_NAME_ANNDATA = "Preprocess_Anndata_File_scGPT"
PATH_ANALYSIS_OUTPUT_ANNDATA = os.path.join( PATH_EXPERIMENT_OUTPUT, ANALYSIS_NAME, EXTRA_ANALYSIS_NAME_ANNDATA)

PATH_OUTPUT_FILE_ANNDATA = os.path.join( PATH_ANALYSIS_OUTPUT_ANNDATA, "cross_tissue_immune_cell_Preprocess.h5ad")
os.makedirs(os.path.dirname(PATH_OUTPUT_FILE_ANNDATA), exist_ok = True)

# Constant to filter the minimum number of cells per cell type for annotation
MINIMAL_NUMBER_CELL_BY_TYPE = 40

In [3]:
# Read the reference/raw data
dataset_anndata = sc.read_h5ad(PATH_INPUT_FILE)

In [4]:
# Look to see if a column exists containing the name of the cell type. If this column exists, it must be written correctly. It must be "celltype".
dataset_anndata.obs.columns

Index(['donor_id', 'Predicted_labels_CellTypist', 'Majority_voting_CellTypist',
       'Majority_voting_CellTypist_high', 'Manually_curated_celltype',
       'assay_ontology_term_id', 'cell_type_ontology_term_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'self_reported_ethnicity_ontology_term_id', 'is_primary_data',
       'organism_ontology_term_id', 'sex_ontology_term_id',
       'tissue_ontology_term_id', 'suspension_type', 'tissue_type',
       'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue',
       'self_reported_ethnicity', 'development_stage', 'observation_joinid'],
      dtype='object')

In [5]:
# Rename the column cell_type by celltype (In this case : rename cell_type by celltype)
dataset_anndata.obs.rename(columns={"cell_type" : "celltype"}, inplace=True)

##### We check whether a column in "var" exists and contains the names of the genes ans if it's in index

In [6]:
dataset_anndata.var

,gene_symbols,feature_is_filtered,feature_name,feature_reference,feature_biotype,feature_length
ENSG00000243485,MIR1302-2HG,False,MIR1302-2HG,NCBITaxon:9606,gene,1021
ENSG00000237613,FAM138A,True,FAM138A,NCBITaxon:9606,gene,1219
ENSG00000186092,OR4F5,True,OR4F5,NCBITaxon:9606,gene,2618
ENSG00000238009,AL627309.1,False,ENSG00000238009.6,NCBITaxon:9606,gene,3726
ENSG00000239945,AL627309.3,False,ENSG00000239945.1,NCBITaxon:9606,gene,1319
...,...,...,...,...,...,...
ENSG00000277836,AC141272.1,False,ENSG00000277836.1,NCBITaxon:9606,gene,288
ENSG00000278633,AC023491.2,False,ENSG00000278633.1,NCBITaxon:9606,gene,2404
ENSG00000276017,AC007325.1,False,ENSG00000276017.1,NCBITaxon:9606,gene,2404
ENSG00000278817,AC007325.4,False,ENSG00000278817.1,NCBITaxon:9606,gene,1213


In [7]:
# If this is not the case, we copy the previous index and put it in a column to avoid losing information.
dataset_anndata.var["index_column"] = dataset_anndata.var.index

# We repalce the column name by gene_name to be understood by scGPT
dataset_anndata.var.rename(columns={"feature_name" : "gene_name"}, inplace=True)

# Move the "gene_name" column to index. (This overwrites the previous column)
dataset_anndata.var = dataset_anndata.var.set_index("gene_name")

# We also need the column gene_name inside the column and not only index
dataset_anndata.var["gene_name"] = dataset_anndata.var.index

/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:767: UserWarning: 
AnnData expects .var.index to contain strings, but got values like:
    ['MIR1302-2HG', 'FAM138A', 'OR4F5', 'ENSG00000238009.6', 'ENSG00000239945.1']

    Inferred to be: categorical

  value_idx = self._prep_dim_index(value.index, attr)


In [9]:
# Loop over the cell types to create fold groups in each of them
for celltype in dataset_anndata.obs.celltype.unique():
    # Retrieve the list of cells of the current cell type
    cell_name_list = list(dataset_anndata.obs.index[dataset_anndata.obs.celltype == celltype])
    # Monitors whether the number of cells within the cell type is sufficient
    if len(cell_name_list) < MINIMAL_NUMBER_CELL_BY_TYPE :
        # Remove the all the cell from the cell type
        print("Warning: The cell type ", celltype, " is removed from the dataset because it does not contain enough cells (", MINIMAL_NUMBER_CELL_BY_TYPE, ").", sep='')
        dataset_anndata = dataset_anndata[~dataset_anndata.obs.celltype.str.contains(celltype)]

In [15]:
# We write a new file containing the pre-processed data. 
dataset_anndata.write_h5ad(PATH_OUTPUT_FILE_ANNDATA)

## R MATRIX : For selection variable gene

In [10]:
# Create the file output
from scipy import io

EXTRA_ANALYSIS_NAME_MATRIX = "Matrix_Files"
PATH_OUTPUT_FILE_MATRIX = os.path.join(PATH_EXPERIMENT_OUTPUT, ANALYSIS_NAME, EXTRA_ANALYSIS_NAME_MATRIX)

# Output path of the pre processed dataset
os.makedirs(PATH_OUTPUT_FILE_MATRIX, exist_ok = True)

In [11]:
# Keep the bacrodes name (so name of cell)
with open(os.path.join(PATH_OUTPUT_FILE_MATRIX) + '/barcodes.tsv', 'w') as f:
    for item in dataset_anndata.obs_names:
        f.write(item + '\n')

In [12]:
# Keep the feture
with open(os.path.join(PATH_OUTPUT_FILE_MATRIX) + '/features.tsv', 'w') as f:
    for item in ['\t'.join([x,x,'Gene Expression']) for x in dataset_anndata.var_names]:
        f.write(item + '\n')

In [20]:
io.mmwrite(os.path.join(PATH_OUTPUT_FILE_MATRIX) + '/matrix.mtx', dataset_anndata.X.T)

#### Need to do a gzip of every file currently in the reperotry (before enter the cell above) | (gzip root_file/Matrix_Files/*)

In [13]:
dataset_anndata.obs.to_csv(os.path.join(PATH_OUTPUT_FILE_MATRIX) + '/metadata.csv') 